# Make local embeddings for each document
- global embeddings meant all articles were clustered together. let's try local

In [1]:
import pickle
import pandas as pd, numpy as np

import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [2]:
### load df

DATA_PATH = './data/cleaned/'
DF_NAME = 'concatenated_df_cleaned_glove.pkl'

with open(DATA_PATH + DF_NAME, 'rb') as f:
    df = pickle.load(f)

In [3]:
class TaggedDocumentIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=doc, tags=[self.labels_list[idx]])

In [4]:
df.head()

,created_at,user,id_str,retweeted_status,Hash words,link,entities,Topic Label,urls,no_link,...,month_year,week_month_year,user_id,cluster,is_retweet,hashtags,link_shorteners,article_text,article_text_cleaned,glove_avg
0,2020-01-31 23:08:33+00:00,"{'id': 19031057, 'id_str': '19031057', 'name':...",1223382589689356288,NaN,#coronavirus,https://twitter.com/user/status/12233825896893...,"{'hashtags': [{'text': 'coronavirus', 'indices...","(43, 0.999852366207409)",[https://www.kron4.com/news/national/when-will...,False,...,2020-01,5,19031057,43,False,[coronavirus],[False],"KRON4 by: Aubree Gordon, University of Michig...","[kron, aubree, gordon, university, michigan, c...","[0.14527764461177867, 1.301592749657575, 0.487..."
1,2020-01-31 23:11:29+00:00,"{'id': 798925214, 'id_str': '798925214', 'name...",1223383328843280384,{'created_at': 'Fri Jan 31 20:41:20 +0000 2020...,#coronavirus #vaccine:,https://twitter.com/user/status/12233833288432...,"{'hashtags': [{'text': 'Coronavirus', 'indices...","(6, 0.9883827140131066)",[https://www.cnbc.com/2020/01/31/coronavirus-w...,False,...,2020-01,5,798925214,6,True,"[Coronavirus, vaccine]",[False],U.S. and international health officials are s...,"[international, health, officials, speeding, w...","[0.4456857605208643, 1.4679776202002541, 0.697..."
2,2020-01-31 23:12:03+00:00,"{'id': 806144538049970176, 'id_str': '80614453...",1223383471999127552,NaN,No hashtags,https://twitter.com/user/status/12233834719991...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(74, 0.9999928079220538)",[https://www.businessinsider.com/australia-suc...,False,...,2020-01,5,806144538049970176,74,False,[None],[False],A leading-edge research firm focused on digit...,"[leading_edge, research, firm_focused, active,...","[-0.012559317511040717, 0.5841067982127425, 0...."
3,2020-01-31 23:35:42+00:00,"{'id': 61298849, 'id_str': '61298849', 'name':...",1223389423045206016,{'created_at': 'Tue Jan 28 18:26:16 +0000 2020...,No hashtags,https://twitter.com/user/status/12233894230452...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(36, 0.9999993854913211)",[https://www.greenmedinfo.com/blog/examining-r...,False,...,2020-01,5,61298849,36,True,[None],[False],Mr. Kennedy is in very safe territory by rep...,"[mr, kennedy, safe, territory, reporting, cdc,...","[1.2950098205753602, 3.404326125156658, 2.0269..."
4,2020-01-31 23:37:23+00:00,"{'id': 1152822375567654912, 'id_str': '1152822...",1223389844719599616,NaN,No hashtags,https://twitter.com/user/status/12233898447195...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(74, 0.9999985797901156)",[https://www.dailymail.co.uk/news/article-7952...,False,...,2020-01,5,1152822375567654912,74,False,[None],[True],By Kylie Stevens and Stephen Gibbs and Nic Wh...,"[kylie, stevens, stephen, gibbs, nic, white, d...","[0.23033713073586115, 4.3884405878634425, 4.87..."


In [10]:
DICT_NAME = 'articles_dict.pkl'
with open(DATA_PATH + DICT_NAME, 'rb') as f:
    articles_dict = pickle.load(f)
    
# CORPUS_NAME = 'articles_corpus.pkl'
# with open(DATA_PATH + CORPUS_NAME, 'rb') as f:
#     articles_corpus = pickle.load(f)

    


print(len(articles_dict))
articles_dict.filter_extremes(no_below=25, no_above=0.5, keep_n=116000)
print(len(articles_dict))
articles_dict.compactify()
print(len(articles_dict))

# new_articles_corpus = make_bow_corpus(df['article_text_cleaned'].tolist(), articles_dict)

116090
9383
9383


In [11]:
# str, id
# articles_dict.token2id
# id: count
# articles_dict.cfs

word_freq = {}
for key, val in articles_dict.token2id.items():
    word_freq[key] = articles_dict.cfs[val]

In [26]:
less_words = [key for key, val in word_freq.items()]

In [33]:
# slim_texts = [[word for word in document if word in less_words] for document in df['article_text_cleaned']]

# or this?...
df['article_text_cleaned_slim'] = df['article_text_cleaned'].apply(lambda x: [word for word in x if word in less_words])



In [34]:
df.head()

,created_at,user,id_str,retweeted_status,Hash words,link,entities,Topic Label,urls,no_link,...,week_month_year,user_id,cluster,is_retweet,hashtags,link_shorteners,article_text,article_text_cleaned,glove_avg,article_text_cleaned_slim
0,2020-01-31 23:08:33+00:00,"{'id': 19031057, 'id_str': '19031057', 'name':...",1223382589689356288,NaN,#coronavirus,https://twitter.com/user/status/12233825896893...,"{'hashtags': [{'text': 'coronavirus', 'indices...","(43, 0.999852366207409)",[https://www.kron4.com/news/national/when-will...,False,...,5,19031057,43,False,[coronavirus],[False],"KRON4 by: Aubree Gordon, University of Michig...","[kron, aubree, gordon, university, michigan, c...","[0.14527764461177867, 1.301592749657575, 0.487...","[gordon, university, michigan, started, wuhan,..."
1,2020-01-31 23:11:29+00:00,"{'id': 798925214, 'id_str': '798925214', 'name...",1223383328843280384,{'created_at': 'Fri Jan 31 20:41:20 +0000 2020...,#coronavirus #vaccine:,https://twitter.com/user/status/12233833288432...,"{'hashtags': [{'text': 'Coronavirus', 'indices...","(6, 0.9883827140131066)",[https://www.cnbc.com/2020/01/31/coronavirus-w...,False,...,5,798925214,6,True,"[Coronavirus, vaccine]",[False],U.S. and international health officials are s...,"[international, health, officials, speeding, w...","[0.4456857605208643, 1.4679776202002541, 0.697...","[international, officials, speeding, work, cre..."
2,2020-01-31 23:12:03+00:00,"{'id': 806144538049970176, 'id_str': '80614453...",1223383471999127552,NaN,No hashtags,https://twitter.com/user/status/12233834719991...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(74, 0.9999928079220538)",[https://www.businessinsider.com/australia-suc...,False,...,5,806144538049970176,74,False,[None],[False],A leading-edge research firm focused on digit...,"[leading_edge, research, firm_focused, active,...","[-0.012559317511040717, 0.5841067982127425, 0....","[leading_edge, firm_focused, active, since, sc..."
3,2020-01-31 23:35:42+00:00,"{'id': 61298849, 'id_str': '61298849', 'name':...",1223389423045206016,{'created_at': 'Tue Jan 28 18:26:16 +0000 2020...,No hashtags,https://twitter.com/user/status/12233894230452...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(36, 0.9999993854913211)",[https://www.greenmedinfo.com/blog/examining-r...,False,...,5,61298849,36,True,[None],[False],Mr. Kennedy is in very safe territory by rep...,"[mr, kennedy, safe, territory, reporting, cdc,...","[1.2950098205753602, 3.404326125156658, 2.0269...","[mr, kennedy, safe, territory, reporting, cdc,..."
4,2020-01-31 23:37:23+00:00,"{'id': 1152822375567654912, 'id_str': '1152822...",1223389844719599616,NaN,No hashtags,https://twitter.com/user/status/12233898447195...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(74, 0.9999985797901156)",[https://www.dailymail.co.uk/news/article-7952...,False,...,5,1152822375567654912,74,False,[None],[True],By Kylie Stevens and Stephen Gibbs and Nic Wh...,"[kylie, stevens, stephen, gibbs, nic, white, d...","[0.23033713073586115, 4.3884405878634425, 4.87...","[stephen, white, daily, mail, australia, publi..."


In [35]:
def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
        
# tagged_docs = list(create_tagged_document(df['article_text_cleaned'].tolist()))
tagged_docs = list(create_tagged_document(df['article_text_cleaned_slim']))

print(type(tagged_docs))
print(tagged_docs[6])



<class 'list'>
TaggedDocument(['paris', 'afp', 'manufacturer', 'ready', 'use', 'middle', 'despite', 'intensive', 'global', 'effort', 'biotech', 'executive', 'told', 'afp', 'friday', 'stephane_bancel', 'chief_executive', 'moderna', 'therapeutics', 'several', 'entities', 'involved', 'international', 'effort', 'create', 'soon', 'possible', 'deadly', 'sars', 'like', 'known', 'ncov', 'already', 'killed', 'moderna', 'working', 'coordination', 'university', 'queensland', 'australia', 'pursuing', 'alternative', 'tracks', 'received', 'funding', 'coalition', 'international', 'body', 'established', 'finance', 'costly', 'biotechnology', 'bancel', 'warned', 'manufacturer', 'able', 'ready', 'summer', 'french', 'language', 'interview', 'afp', 'bancel', 'described', 'technology', 'moderna', 'using', 'approach', 'based', 'messenger_rna', 'technology', 'messenger_rna', 'mrna', 'coded', 'information', 'molecule', 'master', 'technology', 'moderna', 'website', 'explains', 'mrna', 'medicines', 'sets', 'inst

In [37]:
### make model 
model = Doc2Vec(vector_size=100,
                 window=5,
                 min_count=20,
                 workers=2,
                 epochs=100,
                 dm=1,        #1 = paragraph vector - distributed memory; 0 = dbow
                 seed=42)




# Build the Volabulary
# model.build_vocab_from_freq(word_freq)
model.build_vocab(tagged_docs)


In [38]:

# Train the Doc2Vec model
model.train(tagged_docs, total_examples=model.corpus_count, epochs=model.epochs)

In [39]:
MODEL_PATH = './models/'
model_name = 'd2v_pv_slim.model'

model.save(MODEL_PATH + model_name)

In [40]:
model.infer_vector(df['article_text_cleaned'][0]) # one document as a list of strings

array([-4.2293462e-01, -1.0072895e-02, -4.5697591e-01, -1.4392960e+00,
        5.5295724e-01, -4.0921387e-01, -2.5980089e+00,  4.8883688e-01,
       -1.6140534e-01,  1.8301418e+00,  1.6373938e-01, -4.8185714e-02,
        3.1130729e+00,  3.6149103e-02,  1.4827497e+00,  4.1148534e+00,
       -1.3038294e+00, -4.1986480e+00, -5.7112777e-01, -3.3024278e-01,
        2.8320725e+00,  1.6711109e+00, -6.6316199e-01, -2.7718921e+00,
        2.2509916e+00, -3.1317353e+00, -7.5619709e-01,  1.3493427e+00,
       -2.1730457e-01,  1.0624852e+00,  1.0300431e+00,  2.8368437e-03,
       -3.5102115e+00, -3.5257736e-01,  4.7677550e-01, -2.6920221e+00,
        1.9974320e+00, -1.1648186e+00, -5.8281922e-01,  2.8869754e-01,
       -9.5962071e-01, -1.5678179e-02,  1.2789561e-01,  5.4694468e-01,
       -8.9951408e-01, -1.1984601e+00, -7.4158400e-01,  2.3716810e+00,
       -1.4871744e+00,  2.8312728e-01,  3.3893373e+00, -1.7332066e+00,
       -2.0095396e-01, -1.5303822e+00, -2.2315001e+00,  7.0391369e-01,
      

In [41]:
model.infer_vector(df['article_text_cleaned_slim'][0]) # one document as a list of strings

array([-0.5487765 ,  0.18110913, -0.31185022, -1.1884164 ,  0.39214396,
       -0.42236578, -2.7948856 ,  0.53678155, -0.15375412,  1.8303446 ,
        0.11547591, -0.02339075,  3.3737864 , -0.11788098,  1.5224533 ,
        4.3195157 , -1.4830948 , -4.194387  , -0.40817305, -0.54213536,
        2.8654447 ,  1.9219872 , -0.5994078 , -2.5149982 ,  2.2279685 ,
       -3.2216692 , -0.6494393 ,  1.3156669 , -0.17878346,  1.1868769 ,
        1.2163776 , -0.13245864, -3.2883987 , -0.5689883 ,  0.56832737,
       -2.8041983 ,  2.0395374 , -1.2601554 , -0.8071579 ,  0.30416167,
       -0.9728968 ,  0.12640923, -0.16889045,  0.42684728, -0.747363  ,
       -1.389525  , -0.6614712 ,  2.2325165 , -1.6071023 ,  0.43135878,
        3.4209516 , -1.8824617 ,  0.04485067, -1.4709744 , -2.1845536 ,
        0.83487076,  1.6789902 ,  1.1878648 ,  0.6414924 ,  0.34794936,
        0.6620197 ,  0.0983832 , -0.08245859, -0.46630356, -1.5091953 ,
       -3.5166378 , -0.75578284, -1.5450221 , -5.9952254 , -2.74

In [42]:
df['d2v_slim_on_full_text'] = df['article_text_cleaned'].apply(lambda x: model.infer_vector(x))
df['d2v_slim_on_slim_text'] = df['article_text_cleaned_slim'].apply(lambda x: model.infer_vector(x))

In [44]:
### save df

DATA_PATH = './data/cleaned/'
DF_NAME = 'concatenated_df_cleaned_glove_and_d2v_slim.pkl'

with open(DATA_PATH + DF_NAME, 'wb') as f:
    pickle.dump(df, f)